# Homework 3: Word Embeddings
In this homework, we will try to approximate a Skip-gram word embedding via positive pointwise mutual information (PPMI) and truncated singular value decomposition (SVD). 

## The setup
Let's import the required libraries and load the data for preparing our word vectors. We are going to load a list of movie plot summaries (http://www.cs.cmu.edu/~ark/personas/) and use that as our corpus. You do not need to modify them.

In [0]:
# This code gets the data file from github and imports them into Colab
%%capture
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_3/plot_summaries_tokenized.csv

In [3]:
import numpy as np
import pandas as pd
from string import punctuation
from collections import Counter, defaultdict
from math import log2
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds

def load_data():
    """
    Loads the data and returns tokenized summaries.
    
    :return summaries_tokenized: a list that contains tokenized summaries text
    """
    df = pd.read_csv("plot_summaries_tokenized.csv")
    summaries_tokenized = list(df['SUMMARY'].apply(lambda text: text.split()))
    return summaries_tokenized

summaries = load_data()
num_summaries = len(summaries)
print("There are {} summaries.".format(num_summaries))
print("Example tokenized summary:", summaries[1])

There are 42303 summaries.
Example tokenized summary: ['The', 'nation', 'of', 'Panem', 'consists', 'of', 'a', 'wealthy', 'Capitol', 'and', 'twelve', 'poorer', 'districts', 'As', 'punishment', 'for', 'a', 'past', 'rebellion', 'each', 'district', 'must', 'provide', 'a', 'boy', 'and', 'girl', 'between', 'the', 'ages', 'of', '12', 'and', '18', 'selected', 'by', 'lottery', 'for', 'the', 'annual', 'Hunger', 'Games', 'The', 'tributes', 'must', 'fight', 'to', 'the', 'death', 'in', 'an', 'arena', 'the', 'sole', 'survivor', 'is', 'rewarded', 'with', 'fame', 'and', 'wealth', 'In', 'her', 'first', 'Reaping', '12yearold', 'Primrose', 'Everdeen', 'is', 'chosen', 'from', 'District', '12', 'Her', 'older', 'sister', 'Katniss', 'volunteers', 'to', 'take', 'her', 'place', 'Peeta', 'Mellark', 'a', 'bakers', 'son', 'who', 'once', 'gave', 'Katniss', 'bread', 'when', 'she', 'was', 'starving', 'is', 'the', 'other', 'District', '12', 'tribute', 'Katniss', 'and', 'Peeta', 'are', 'taken', 'to', 'the', 'Capitol',

We have ~42000 summaries containing ~13000000 words. We will now proceed by creating a vocabulary and will limit its size to something computationally feasible. You may find python's collections.Counter function useful. You may not import any additional libraries.

# 1. Create Vocabulary
We will start from creating our vocabulary. Vocabulary contains unigrams and their counts.

In [0]:
###################
# Do not modify
###################
min_count = (1 / 100) * len(summaries) #min is a hundreth of the num_summaries - 423
max_count = (1 / 10) * len(summaries) #max is a tenth of the num_summaries - 4230

In [0]:
def creat_vocabulary(tokenized_documents, min_count, max_count):
    #does this take in just one document for one movie or all?
    """
    This function takes in tokenized documents and returns a
    vocabulary and word <-> index lookup dictionary of some frequently appearing words.
    
    :param tokenized_documents: a list of tokenized strings
    :param min_count: minimum unigram count
    :param max_count: maximum unigram count
    :return vocab: a Counter where vocab[word] = count of word's occurences in all the documents
    :return word2idx: a word -> index lookup Dictionary for words in vocab.
    :return idx2word: a index -> word lookup Dictionary for words in vocab.
    """
    # 1a. Compute unigram counts. A unigram is a single word, e.g. foo. BAG OF WORDS
    #vocab = Counter()
    #print(tokenized_documents[:10])
    #vocab_movie = Counter()

    vocab = Counter()
    for movie in tokenized_documents:
      for token in movie:
        vocab[token] += 1

    print('%d vocabs before' % len(vocab))
    #for movie in tokenized_documents:
      #vocab = dict(vocab) #turn vocab into regular dictionary
      #vocab_movie = Counter(movie) #counts occur of tokens in a movie
      #vocab = vocab_movie + vocab

    # 1b. Remove unigrams that have #(unigram) < min_count or #(unigram) > max_count
    # to eliminate unigrams occurring very frequently or infrequently. 
    # This will limit its size to something computationally feasible.
    '''
    for movie in tokenized_documents:
      vocab_movie = Counter(movie)
      for word in vocab_movie:
        vocab[word] += vocab_movie[word]
    '''
    for unigram in list(vocab.items()):
      if unigram[1] < min_count or unigram[1] > max_count:
        del vocab[unigram[0]]
    print('%d vocabs after' % len(vocab))
    '''
    vocab1 = vocab
    for unigram,count in vocab1:
      if count < min_count or count > max_count:
        del vocab1[unigram]
    '''
    #for unigram in unigram_counts:
  #delete a word and its count if the # of times the word appears in total is > max unigram count
      #if (unigram_counts[unigram] < min_count) or (unigram_counts[unigram] > max_count):
        #unigram_counts = remove_key(unigram_counts,unigram)
    
   
          
    # 1c. Build word <-> index lookup for words in vocab.
    word2idx, idx2word = {}, {}
    index = 0
    for unigram in vocab.items(): #each unigram is a tuple of ("go":5), unigram[0] = "go", unigram[1] = 5
      word2idx[unigram[0]] = index
      index += 1
      
    index = 0 #reset index so all are matching
    
    for unigram in vocab.items():
      idx2word[index] = unigram[0]
      index+=1
    print(word2idx["despite"])
    print(idx2word[3])


    return vocab, word2idx, idx2word

In [0]:
def remove_key(dictionary, key):
    r = dict(dictionary)
    del r[key]
    return r

In [7]:
vocab, word2idx, idx2word = creat_vocabulary(summaries, min_count, max_count)

214147 vocabs before
2750 vocabs after
3
despite


# 2. Build Term-Context Matrix

In [0]:
###################
# Do not modify
###################
window_size = 3

In [0]:
def build_term_context_matrix(tokenized_documents, vocab, window_size):
    """
    This function returns a `word_pair_count` Counter with each 
    word_pair_count[(w, c)] = number of times the word `c` occurs in the context of word `w`. (where `w`, `c` belong to the vocab)
    To make it efficient, instead of building the sparse term-context matrix, 
    we will build 3 separate Counters: word_pair_count, w_count, c_count
    You may find python's Counter useful here

    :param tokenized_documents: a list of tokenized strings
    :param vocab: vocabulary Counter
    :param window_size: context window size
    :return word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window, i.e. #(w, c)
    :return w_count: a Counter where w_count[w] = the number of times w occured in the documents, i.e. #(w)
    :return c_count: a Counter where c_count[c] = the number of times c occured in the documents, i.e. #(c)
    """
    word_pair_count = Counter()  
    w_count = Counter()
    c_count = Counter()
    target = ""
   
    count = 0
    for movie in tokenized_documents:
      for index in range(len(movie)):
        target = movie[index]
        if target in vocab: #if target not in vocab, go to next index for a target
          w_count[target] += 1
          for step in range(1,window_size+1):
            if (index-step) >= 0:
              if movie[index-step] in vocab:
                #context_left.append(movie[index-step])
                c_count[movie[index-step]] += 1
                word_pair_count[(target,movie[index-step])] += 1
            if (index+step) <= (len(movie)-1):
              if movie[index+step] in vocab:
                #context_right.append(movie[index+step])
                c_count[movie[index+step]] += 1
                word_pair_count[(target,movie[index+step])] += 1
          '''
          context_words.append(context_left)
          context_words.append(context_right)
          for c in context_words:
            word_pair_count[(target,c)] += 1
          '''



          '''
          if index-step => 0 and index+step <= (len(movie)-1):
            if movie[index-step] in vocab:
              context_words.append(movie[index-step])
              c_count[movie[index-step]] += 1
              word_pair_count[(target,movie[index-step])] += 1
            if movie[index+step] in vocab:
              context_words.append(movie[index+step])
              c_count[movie[index+step]] += 1
              word_pair_count[(target,movie[index+step])] += 1
          '''
          '''

          for window_location in range(1,window_size+1):
            if (index - window_location) < 0:
              #if movie[index+window_location] not in vocab:
                #continue
              #else:
              context_words.append(movie[index+window_location])
              c_count[movie[index+window_location]] += 1
              #continue
            if (index + window_location) > (len(movie)-1):
              #if movie[index-window_location] not in vocab:
                #continue
              #else:
              context_words.append(movie[index-window_location])
              c_count[movie[index-window_location]] += 1
              #continue
            else:
              context_words.append(movie[index-window_location]) #left
              context_words.append(movie[index+window_location]) #right
          for c in context_words:
            word_pair_count[(target,c)] += 1
    '''        
    return word_pair_count, w_count, c_count

In [10]:
word_pair_count

NameError: ignored

In [11]:
word_pair_count, w_count, c_count = build_term_context_matrix(summaries, vocab, window_size)
print("There are {} word-context pairs".format(len(word_pair_count)))

# The number of w_count and c_count should match your number of vocab
assert len(w_count) == len(vocab)
assert len(c_count) == len(vocab)

There are 1917545 word-context pairs


# 3. Build Positive Pointwise Mutual Information (PPMI) Matrix
In this part, you will build a PPMI matrix using Scipy's Compressed Sparse Column matrix to save storage space. (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html)

Sparse matrix is a matrix which contains very few non-zero elements. When a sparse matrix is represented with a 2-dimensional array, we waste a lot of space to represent that matrix. In NLP application, it's quite common to use sparse matrix since the size of vocabulary is usually very large. 

Below is an example of how to build a sparse matrix where `data`, `row` and `col` satisfy the relationship `M[row[k], col[k]] = data[k]`.

```python
>>> row = np.array([0, 2, 2, 0, 1, 2])
>>> col = np.array([0, 0, 1, 2, 2, 2])
>>> data = np.array([1, 2, 3, 4, 5, 6])
>>> M = csc_matrix((data, (row, col)))
>>> M.toarray()
array([[1, 0, 4],
       [0, 0, 5],
       [2, 3, 6]])
```

Recall that
$$
\begin{gather*}
  \text{PMI}(w, c) = \log_2 \frac{P(w, c)}{P(w)P(c)} \\
  \text{PPMI}(w, c) = \max(0, \text{PMI}(w, c))
\end{gather*}
$$
You should use `log2` function from the math package that is alreadly imported for you.

In [0]:
def build_PPMI_matrix(word_pair_count, w_count, c_count, word2idx):
    """
    This function returns a PPMI matrix represented by a csc sparse matrix.

    :params word_pair_count: a Counter where word_pair_count[(w, c)] = count of c's occurences in w's context window
    :return w_count: a Counter where w_count[w] = the number of times w occured in the documents
    :return c_count: a Counter where c_count[c] = the number of times c occured in the documents
    :return word2idx: a word -> index lookup Dictionary for words in vocab
    :return PPMI: PPMI csc sparse matrix
    """
    data, rows, cols = [], [], []
    #data is ppmi for each pair of row and col
    total_occurences = sum(word_pair_count.values())
    for (w, c), n in word_pair_count.items():
      rows.append(word2idx[w])
      cols.append(word2idx[c])
      top = word_pair_count[(w,c)]/total_occurences
      bottom = (c_count[c]/total_occurences)*(w_count[w]/total_occurences)
      pmi = top/bottom
      pmi = log2(pmi)
      ppmi = max(0,pmi)
      data.append(ppmi)
    PPMI = csc_matrix((data, (rows, cols)))
    return PPMI

In [15]:
PPMI.shape

(2750, 2750)

In [16]:
sum(word_pair_count.values())

3872596

In [0]:
PPMI = build_PPMI_matrix(word_pair_count, w_count, c_count, word2idx)

# The shape of PPMI matrix should match your number of vocab
assert PPMI.shape == (len(vocab), len(vocab))

# 4. Truncated SVD
In this part, we will obtain a dense low-dimensional vectors via truncated (rank-k) SVD. You should use `svds` function from Sicpy that is already imported for you to obtain the SVD factorization.
(https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html)


In [0]:
###################
# Do not modify
###################
rank = 20

In [0]:
def get_embeddings(PPMI, rank):
    """
    Reutrns the left singular vectors as word embeddings via truncated SVD

    :params PPMI: PPMI csc sparse matrix
    :params rank: number of singular values and vectors to compute
    :return u: left sigular vectors from sprase SVD
    :return s: the singular values from sparse SVD
    """
    u,s,vt = svds(PPMI, rank)
    return u, s

In [0]:
'''
embeddings, _ = get_embeddings(PPMI, rank)

# The shape of the embeddings matrix should be (# vocab, rank)
assert embeddings.shape == (len(vocab), rank)
'''
embeddings, _ = get_embeddings(PPMI, rank)
embeddings /= np.linalg.norm(embeddings, axis=1, keepdims=True)  # Normalize embeddings matrix

# The shape of the embeddings matrix should be (# vocab, rank)
assert embeddings.shape == (len(vocab), rank)

# Make sure embeddings is normalized
assert True == np.isclose(np.linalg.norm(embeddings[0]), 1)

# 5. Evaluate Word Embeddings via Cosine Similarity

Using cosine similarity as a measure of distance [§6.4 Jurafsky & Martin](https://web.stanford.edu/~jurafsky/slp3/6.pdf), we will now find the closest words to a certain word. We define cosine similarity as, $$cosine(\overrightarrow{v},\overrightarrow{w}) = \frac{\overrightarrow{v} \cdot \overrightarrow{w}}{\vert v \vert \vert w \vert}$$

Please complete the function below that calculates the 'K' closest words from the vocabulary. You may not use any additional libraries.

In [0]:
###################
# Do not modify
###################
num_neighbors = 5

In [0]:
def cosine_distances(matrix, vector):
    """
    The function takes in a matrix and a vector (both normalized) 
    and returns the cosine distances for this vector against all others.
    The pretrained embeddings are normalized.

    :params matrix: word embeddings matrix
    :params vector: word vector for a particular word
    :return distances: a cosine distances vector
    """
    distances = []
    for row in matrix:
      top = np.dot(vector,row)
      bottom = np.linalg.norm(vector)*np.linalg.norm(row)
      cosine = top/bottom
      distances.append(cosine)

    
    return  distances


def nearest_neighbors(embeddings, word, k, word2idx, idx2word):
    """
    For each query word, this function returns the k closest words from the vocabulary.

    :params embeddings: word embedding matrix
    :params word: query word
    :params k: number of closest words to return
    :params word2idx: a word -> index lookup dictionary
    :params idx2word: a index -> word lookup dictionary
    :return nearest_neighbors: a list of cloest words
    """
    vector = embeddings[word2idx[word]]
    distances = cosine_distances(embeddings, vector)
    #nearest_neighbors = []
    
    rank_distances = np.argsort(distances)[-(k+1):][::-1]
    #index = 0
    nearest_neighbors = []
    for d in range(len(rank_distances)):
      if idx2word[rank_distances[d]] == word:
        continue
      else:
        nearest_neighbors.append(idx2word[rank_distances[d]])


    return nearest_neighbors

In [22]:
query_words = ["doctor", "zombie", "robot", "eat", "bus"]
for word in query_words:
    print(word, nearest_neighbors(embeddings, word, num_neighbors, word2idx, idx2word))

doctor ['Anna', 'priest', 'nurse', 'teacher', 'Elizabeth']
zombie ['infected', 'creatures', 'zombies', 'vampires', 'possessed']
robot ['alien', 'weapon', 'demon', 'creature', 'virus']
eat ['throw', 'sleep', 'stand', 'wait', 'wear']
bus ['road', 'boat', 'truck', 'station', 'airport']


# 6. Evaluate Word Embeddings via Analogous Tasks

The embedding space is known to capture the semantic context of words. An example of it is $\overrightarrow{woman} - \overrightarrow{man} \simeq \overrightarrow{queen} - \overrightarrow{king}$. Use the `cosine_distances()` function you wrote above to find such relations.

In [0]:
def relation(embeddings, query_words, word2idx, idx2word):
    """
    Takes in 3 words and returns the closest word (in terms of cosine similarity)
    to the normalized algebraic addition of the three vectors.
    The parameters follow this order : word_vec1 - word_vec2 ~ closest - word_vec3

    :params embeddings: word embedding matrix
    :params query_words: a list of query words in the following order: [word1, word2, word3]
    :params word2idx: a word -> index lookup dictionary
    :params idx2word: a index -> word lookup dictionary
    :return closet_word: the closest word for the relation
    """
    #want to get vectors from embeddings matrix and then add them together and subtract the one and then get words for the nearest vector
    word1, word2, word3 = query_words
    three_vectors = []
    if all(word in vocab for word in query_words):
      for words in query_words:

        vector = embeddings[word2idx[words]]
        three_vectors.append(vector)
      three_vectors = three_vectors[0] - three_vectors[1] + three_vectors[2]
      three_vectors /= np.linalg.norm(three_vectors)
      distances = cosine_distances(embeddings,three_vectors)
      rank_distances = np.argsort(distances)
      
      #closet_word = rank_distances[len(rank_distances)-1]
      closet_word = idx2word[rank_distances[len(rank_distances)-1]]
      if closet_word == word1 or closet_word == word2 or closet_word == word3:
        closet_word = idx2word[rank_distances[len(rank_distances)-2]]
        

      return closet_word
    else:
      missing = [w for w in query_words if w not in vocab]
      raise Exception("missing {} from vocabulary".format(", ".join(missing)))

In [24]:
queries = [["doctor", "nurse", "king"], ["robot", "weapon", "bus"], ["sing", "song", "justice"], ["elderly", "kids", "teenager"], ["soldier", "wound", "telephone"]]
for query in queries:
  closet_word = relation(embeddings, query, word2idx, idx2word)
  print("{} - {} ~= {} - {}".format(query[0], query[1], closet_word, query[2]))

doctor - nurse ~= Lord - king
robot - weapon ~= road - bus
sing - song ~= defend - justice
elderly - kids ~= widow - teenager
soldier - wound ~= post - telephone
